In [22]:
#顧客情報から融資できるか予測
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import warnings
import mglearn
# 実行上問題ない注意は非表示にする
warnings.filterwarnings('ignore') 

In [23]:
#csvファイルの読み込み
df = pd.read_csv("data/bank_01.csv")

In [24]:
#ワンホットエンコーディング
data_dummies = pd.get_dummies(df)
data_dummies.columns
data_dummies.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,day_of_week_fri,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_failure,poutcome_nonexistent,poutcome_success,y_no,y_yes
0,30,487,2,999,0,-1.8,92.893,-46.2,1.313,5099.1,...,1,0,0,0,0,0,1,0,1,0
1,39,346,4,999,0,1.1,93.994,-36.4,4.855,5191.0,...,1,0,0,0,0,0,1,0,1,0
2,25,227,1,999,0,1.4,94.465,-41.8,4.962,5228.1,...,0,0,0,0,1,0,1,0,1,0
3,38,17,3,999,0,1.4,94.465,-41.8,4.959,5228.1,...,1,0,0,0,0,0,1,0,1,0
4,47,58,1,999,0,-0.1,93.200,-42.0,4.191,5195.8,...,0,1,0,0,0,0,1,0,1,0


In [25]:
features = data_dummies.loc[:, 'age':'poutcome_success']
X = features.values
y = data_dummies['y_no'].values
print("X.shape: {}  y.shape: {}".format(X.shape, y.shape))
print(data_dummies.y_no.value_counts())

X.shape: (4119, 63)  y.shape: (4119,)
1    3668
0     451
Name: y_no, dtype: int64


In [32]:
pipe = Pipeline([('scaler', StandardScaler()), ('classifier', SVC())])
param_grid = [
    {'classifier': [SVC()], 'classifier__C': [0.001, 0.01, 0.1, 1]},
    {'classifier': [RandomForestClassifier()], 'scaler': [None], 'classifier__max_features': [1, 2, 3, 4, 5]}
]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, scoring="roc_auc")
grid.fit(X_train, y_train)
print("Best parameters: ", grid.best_params_)
print("grid best score, ", grid.best_score_)
print("Test set accuracy: {:.2f}".format(grid.score(X_test, y_test)))

Best parameters:  {'classifier': SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False), 'classifier__C': 0.1}
grid best score,  0.9249419295996347
Test set accuracy: 0.89


In [33]:
#確認用
#grid.cv_results_ 